In [2]:
import ftplib
ftp = ftplib.FTP('f31-preview.runhosting.com','4009006_DATOS','Rafa9999')

In [3]:
ftp.dir()

drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16  2021 .
drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16  2021 ..
-rw-r--r--   1 4009006_DATOS 1111        33182 Dec 16  2021 books.xml
-rw-r--r--   1 4009006_DATOS 1111         3841 Dec 16  2021 colors.json
-rw-r--r--   1 4009006_DATOS 1111       763509 May  3 21:07 OptaF24.xml


In [4]:
filename = 'OptaF24.xml'
ftp.retrbinary("RETR " + filename, open(filename,'wb').write)

'226 Transfer complete'

In [5]:
ftp.quit()

'221 Goodbye.'

In [6]:
import xml.etree.ElementTree as ET
tree = ET.parse('OptaF24.xml')

In [9]:
gameFile = tree.getroot() 
gameFile.tag

'Games'

In [11]:
gameFile.attrib

{'timestamp': '2012-09-11T10:20:32'}

In [18]:
game = tree.findall('Game')
print('Número de partidos:', len(game))

Número de partidos: 1


In [19]:
eventos = game[0].findall('Event')
print('Número de eventos:',len(eventos))

Número de eventos: 1673


In [43]:
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []
minute = []
second = []
half = []
team = []

In [44]:
for evento in eventos:
    if evento.attrib['type_id']=='1':
        half.append(evento.attrib['period_id'])
        minute.append(evento.attrib['min'])
        second.append(evento.attrib['sec'])
        team.append(evento.attrib['team_id'])
        x_origin.append(evento.attrib['x'])
        y_origin.append(evento.attrib['y'])
        outcome.append(evento.attrib['outcome'])
        
        qualifiers = evento.findall('Q')
        
        for q in qualifiers:
            if q.attrib['qualifier_id']=='140':
                x_destination.append(q.attrib['value'])
            if q.attrib['qualifier_id']=='141':
                y_destination.append(q.attrib['value'])
        

In [47]:
len(x_origin),\
len(y_origin),\
len(x_destination),\
len(y_destination),\
len(outcome),\
len(minute),\
len(second),\
len(half),\
len(team)

(946, 946, 946, 946, 946, 946, 946, 946, 946)

In [51]:
game[0].attrib

{'id': '360481',
 'away_team_id': '43',
 'away_team_name': 'Manchester City',
 'competition_id': '8',
 'competition_name': 'English Barclays Premier League',
 'game_date': '2011-08-21T16:00:00',
 'home_team_id': '30',
 'home_team_name': 'Bolton Wanderers',
 'matchday': '2',
 'period_1_start': '2011-08-21T16:00:38',
 'period_2_start': '2011-08-21T17:03:47',
 'season_id': '2011',
 'season_name': 'Season 2011/2012'}

In [52]:
equipos={game[0].attrib['home_team_id']: game[0].attrib['home_team_name'],
        game[0].attrib['away_team_id']: game[0].attrib['away_team_name']
        }
equipos

{'30': 'Bolton Wanderers', '43': 'Manchester City'}

In [55]:
import pandas as pd

pases = pd.DataFrame({'team':team,
                     'half':half,
                     'minute':minute,
                     'second':second,
                     'x_origin':x_origin,
                     'y_origin':y_origin,
                     'x_destination':x_destination,
                      'y_destination':y_destination,
                      'outcome':outcome
                     })

pases.team.replace(equipos,inplace=True)
pases

team half minute second x_origin y_origin x_destination  \
0     Manchester City    1      0      1     50.1     50.0          52.4   
1     Manchester City    1      0      2     48.2     49.1          29.0   
2     Manchester City    1      0     19     27.8    100.0          49.3   
3    Bolton Wanderers    1      0     23     50.9     20.0          63.6   
4     Manchester City    1      0     30     29.3     73.0          30.2   
..                ...  ...    ...    ...      ...      ...           ...   
941  Bolton Wanderers    2     93     42     16.3     82.5          71.7   
942   Manchester City    2     93     45     28.6     27.6          34.2   
943   Manchester City    2     93     46     34.4      9.1          63.0   
944   Manchester City    2     93     47     62.4     20.7          61.8   
945  Bolton Wanderers    2     94      4      5.0     63.0          68.1   

    y_destination outcome  
0            49.1       1  
1            76.5       1  
2            93.4       0  
3            30.1       1  
4            55.6       1  
..            ...     ...  
941          70.7       0  
942           9.5       1  
943          20.7       1  
944          12.5       1  
945          57.5       1  

[946 rows x 9 columns]

In [59]:
pases.outcome = pases.outcome.astype('int')

In [60]:
pases.outcome = pd.to_numeric(pases.outcome)

In [61]:
# ¿Qué equipo ha dado más pases buenos?
pases[['team','outcome']].groupby('team',as_index=False).sum()

team  outcome
0  Bolton Wanderers      279
1   Manchester City      427

In [67]:
# Pases de más de 20 metros en el eje x
pases['x_distance'] = pases.x_destination.astype('float') - pases.x_origin.astype('float')
pases.loc[(pases.x_distance>20) & (pases.outcome==1)].team.value_counts()

Bolton Wanderers    35
Manchester City     32
Name: team, dtype: int64